In [ ]:
from predict import *
from predict import _metrics_wrt_t, _metrics_t
import pickle
from privacy_policies_dataset import PrivacyPoliciesDataset as PPD
from os.path import join, isfile
from os import listdir
import matplotlib.pyplot as plt

In [ ]:
#We set the folder path containing the models and load the labels
folder = 'trained_models/MulticlassOpt'
models = load_12CNN_model(folder)

#We load the labels
with open('labels.pkl') as f:
    labels = pickle.load(f)

#We set the folder containing the data already prepared for predicting
data_folder = 'datasets'
data_file = join(data_folder, 'test_dataset_label6.pkl')

#We load the data and get just the segments
data = PPD.unpickle_dataset(data_file)

#We predict the labels
predictions = predict(data, models)

#Computation of all metrics 
f1s, ps, rs, ts = _metrics_wrt_t(data.labels_tensor,predictions)
figure = plt.figure(figsize=(18,5))
figure.suptitle('12-CNN Micro Averages with respect t')
ax_f1 = figure.add_subplot(131)
ax_f1.set_ylim(0,1)
ax_p = figure.add_subplot(132)
ax_p.set_ylim(0,1)
ax_r = figure.add_subplot(133)
ax_r.set_ylim(0,1)
ax_f1.plot(ts, f1s)
ax_p.plot(ts, ps)
ax_r.plot(ts, rs)
plt.show()

In [ ]:
labels

In [ ]:
save_metrics(data.labels_tensor, predictions, 'trained_models/MulticlassOpt/metrics.csv')

In [ ]:
ts = best_ts(data.labels_tensor, predictions)

In [ ]:
ts.unsqueeze(0)

In [ ]:
y_hat = (predictions > ts.unsqueeze(0)).double()

In [ ]:
macro_metrics(data.labels_tensor, y_hat)

In [ ]:
micro_metrics(data.labels_tensor, y_hat)